In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

from lib import dpf, panda_models, panda_datasets, panda_training
from lib.utils import file_utils, torch_utils, misc_utils

print(torch.__version__, np.__version__)

1.3.0 1.17.3


In [3]:
# Experiment configuration
experiment_name = "dpf_no_vision"
dataset_args = {
    'use_proprioception': True,
    'use_haptics': True,
    'use_vision': False,
    'vision_interval': 10,
}

In [4]:
# Create models & training buddy

dynamics_model = panda_models.PandaSimpleDynamicsModel(state_noise=(0.05))
measurement_model = panda_models.PandaMeasurementModel(units=32)

pf_model = dpf.ParticleFilterNetwork(dynamics_model, measurement_model)

buddy = torch_utils.TrainingBuddy(
    experiment_name,
    pf_model,
    optimizer_names=["e2e", "dynamics", "measurement"],
    log_dir="logs/pf",
    checkpoint_dir="checkpoints/pf"
)

Using device: cpu
Loaded checkpoint from path: checkpoints/pf/dpf_no_vision-0000000000468693.ckpt


# Dynamics Model Pre-Training

In [5]:
# dynamics_trainset = panda_datasets.PandaDynamicsDataset(
#     "data/pull-test.hdf5",
#     "data/push-test.hdf5",
#     **dataset_args
# )

In [6]:
# dataloader = torch.utils.data.DataLoader(dynamics_trainset, batch_size=256, shuffle=True, num_workers=2)

# for i in range(0):
#     print("Training epoch", i)
#     panda_training.train_dynamics(buddy, pf_model, dataloader, log_interval=10)
#     print()


# Measurement Model Pre-Training

In [7]:
measurement_trainset = panda_datasets.PandaMeasurementDataset(
    ("data/pull-test.hdf5", 1000),
    ("data/push-test.hdf5", 1000),
    std_dev=0.15,
    **dataset_args
)

Progress: [####################] 100.0%


In [8]:
measurement_trainset_loader = torch.utils.data.DataLoader(measurement_trainset, batch_size=64, shuffle=True, num_workers=16)
for i in range(3):
    print("Training epoch", i)
    panda_training.train_measurement(buddy, pf_model, measurement_trainset_loader, log_interval=100)
    print()

# measurement_trainset_loader = torch.utils.data.DataLoader(measurement_trainset, batch_size=64, shuffle=True, num_workers=2)
# for i in range(1):
#     print("Training epoch", i)
#     panda_training.train_measurement(buddy, pf_model, measurement_trainset_loader, log_interval=100)
#     print()

# measurement_trainset_loader = torch.utils.data.DataLoader(measurement_trainset, batch_size=256, shuffle=True, num_workers=2)
# for i in range(1):
#     print("Training epoch", i)
#     panda_training.train_measurement(buddy, pf_model, measurement_trainset_loader, log_interval=100)
#     print()



Training epoch 0
..........[0.55%]..........[1.09%]..........[1.64%]..........[2.18%]..........[2.73%]..........[3.28%]..........[3.82%]..........[4.37%]..........[4.92%].........Saved checkpoint to path: checkpoints/pf/dpf_no_vision-0000000000010000.ckpt
.[5.46%]..........[6.01%]..........[6.56%]..........[7.10%]..........[7.65%]..........[8.20%]..........[8.74%]..........[9.29%]..........[9.83%]..........[10.38%].........Saved checkpoint to path: checkpoints/pf/dpf_no_vision-0000000000020000.ckpt
.[10.93%]..........[11.47%]..........[12.02%]..........[12.57%]..........[13.11%]..........[13.66%]..........[14.21%]..........[14.75%]..........[15.30%]..........[15.84%].........Saved checkpoint to path: checkpoints/pf/dpf_no_vision-0000000000030000.ckpt
.[16.39%]..........[16.94%]..........[17.48%]..........[18.03%]..........[18.58%]..........[19.12%]..........[19.67%]..........[20.22%]..........[20.76%]..........[21.31%].........Saved checkpoint to path: checkpoints/pf/dpf_no_vision-0000

KeyboardInterrupt: 

In [9]:
buddy.save_checkpoint(label="before_e2e_training")

Saved checkpoint to path: checkpoints/pf/dpf_no_vision-before_e2e_training.ckpt


# End-to-end Training

In [13]:
# Create end-to-end dataset

e2e_trainset = panda_datasets.PandaParticleFilterDataset(
    ("data/pull-test.hdf5", 1000),
    ("data/push-test.hdf5", 1000),
    subsequence_length=20,
    particle_count=50,
    particle_variances=(.2,),
    **dataset_args
)

Parsed data: 20570 active, 8605 inactive
Keeping: 8605


In [ ]:
##### Train end-to-end
pf_model.dynamics_model.state_noise = (0.1)

pf_model.freeze_measurement_model = False
pf_model.freeze_dynamics_model = True

# e2e_trainset_loader = torch.utils.data.DataLoader(e2e_trainset, batch_size=32, shuffle=True, num_workers=2)
# for i in range(3):
#     print("Training epoch", i)
#     panda_training.train_e2e(buddy, pf_model, e2e_trainset_loader, loss_type="mse", log_interval=100)

e2e_trainset_loader = torch.utils.data.DataLoader(e2e_trainset, batch_size=64, shuffle=True, num_workers=2)
for i in range(3):
    print("Training epoch", i)
    panda_training.train_e2e(buddy, pf_model, e2e_trainset_loader, loss_type="mse", log_interval=100)

Training epoch 0
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................Saved checkpoint to path: checkpoints/pf/dpf_no_vision-0000000000460000.ckpt
..............................Epoch loss: 0.022260558
Training epoch 2
............................................................................................................................................................................................................................................................................................................................................................................................................................

In [23]:
buddy.save_checkpoint()

Saved checkpoint to path: checkpoints/pf/dpf_no_vision-0000000000468693.ckpt


# Model eval

In [5]:
eval_trajectories_list = []

def load_trajectories(label, validation, include_pull, include_push, max_count=10):
    if validation:
        # Validation set
        files = [
            ("data/pull-test-small.hdf5", max_count),
            ("data/push-test-small.hdf5", max_count)
        ]
    else:
        # Training set
        files = [
            ("data/pull-test.hdf5", max_count),
            ("data/push-test.hdf5", max_count)
        ]
    
    filtered_files = []
    if include_pull:
        filtered_files.append(files[0])
    if include_push:
        filtered_files.append(files[1])

    trajectories = panda_datasets.load_trajectories(
        *filtered_files,
        **dataset_args
    )
    eval_trajectories_list.append((label, trajectories))

load_trajectories("Validation all", validation=True, include_pull=True, include_push=True)
load_trajectories("Validation pull", validation=True, include_pull=True, include_push=False)
load_trajectories("Validation push", validation=True, include_pull=False, include_push=True)

load_trajectories("Training all", validation=False, include_pull=True, include_push=True)
load_trajectories("Training pull", validation=False, include_pull=True, include_push=False)
load_trajectories("Training push", validation=False, include_pull=False, include_push=True)

### Final Model

In [6]:
pf_model.dynamics_model.state_noise = (0.1)

# Evaluate final model
for label, trajectories in eval_trajectories_list:
    print()
    print("###############################")
    print("##", label)
    print("###############################")
    pred, actual = panda_training.rollout(pf_model, trajectories, start_time=0, max_timesteps=1000, particle_count=200, noisy_dynamics=True)
    panda_training.eval_rollout(pred, actual)


###############################
## Validation all
###############################
...................................
Position RMSE, degrees:  7.747230004772526

###############################
## Validation pull
###############################
................................................................
Position RMSE, degrees:  6.278561321414542

###############################
## Validation push
###############################
...................................
Position RMSE, degrees:  10.60692434232473

###############################
## Training all
###############################
.......................................
Position RMSE, degrees:  9.446203702320055

###############################
## Training pull
###############################
............................................................................
Position RMSE, degrees:  4.299478150910205

###############################
## Training push
###############################
...................................

### Model without end-to-end training

In [7]:
# Back up model
buddy.save_checkpoint()

# Load the pre-end-to-end-training version of the model & evaluate
buddy.load_checkpoint(label="before_e2e_training")
for label, trajectories in eval_trajectories_list:
    print()
    print("###############################")
    print("##", label)
    print("###############################")
    pred, actual = panda_training.rollout(pf_model, trajectories, start_time=0, max_timesteps=1000, particle_count=200, noisy_dynamics=True)
    panda_training.eval_rollout(pred, actual)

# Restore model
buddy.load_checkpoint()

Saved checkpoint to path: checkpoints/pf/dpf_no_vision-0000000000468693.ckpt
checkpoints/pf/dpf_no_vision-before_e2e_training.ckpt
Loaded checkpoint from path: checkpoints/pf/dpf_no_vision-before_e2e_training.ckpt

###############################
## Validation all
###############################
...................................
Position RMSE, degrees:  8.17580559217061

###############################
## Validation pull
###############################
................................................................
Position RMSE, degrees:  8.003006928811164

###############################
## Validation push
###############################
...................................
Position RMSE, degrees:  11.023389150550734

###############################
## Training all
###############################
.......................................
Position RMSE, degrees:  18.086457035409655

###############################
## Training pull
###############################
......................